### 나무위키 - 최근 변경내역 말뭉치(corpus) 만들기

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

##### 1. 페이지의 URL 정보 추출하기

In [2]:
url = 'https://namu.wiki/RecentChanges'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [3]:
# //*[@id="QD8sx38qa"]/div[2]/div/article/div[2]/div/div[1]/div/div/table/tbody
# //*[@id="QD8sx38qa"]/div[2]/div/article/div[2]/div/div[1]/div/div/table/tbody/tr
trs = driver.find_elements(By.XPATH, '//*[@id="QD8sx38qa"]/div[2]/div/article/div[2]/div/div[1]/div/div/table/tbody/tr')
len(trs)

112

In [4]:
tr = trs[0]
tr.find_element(By.CSS_SELECTOR, 'td > a').text.strip()

'파일:982xkmb.webp'

In [5]:
tr.find_element(By.CSS_SELECTOR, 'td > a').get_attribute('href')

'https://namu.wiki/w/%ED%8C%8C%EC%9D%BC:982xkmb.webp'

In [6]:
page_urls = []
for tr in trs:
    try:
        page_urls.append(tr.find_element(By.CSS_SELECTOR, 'td > a').get_attribute('href'))
    except:
        continue

In [7]:
len(page_urls)

100

In [8]:
# 중복 URL 제거
page_urls = list(set(page_urls))
len(page_urls)

71

In [9]:
page_urls[:3]

['https://namu.wiki/w/%EC%9C%A0%EA%B8%B0%EC%83%81(%EB%86%8D%EA%B5%AC%EC%84%A0%EC%88%98)',
 'https://namu.wiki/w/spaceuk',
 'https://namu.wiki/w/%EB%9D%BC%EB%94%94%EC%98%A4']

In [10]:
driver.close()

##### 2. URL 페이지 정보를 기반으로 크롤링하기

In [11]:
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(page_urls[10])

In [12]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
title = soup.select_one('.hH5f5Do0').find('h1').get_text().strip()
title

'헤어질 결심/줄거리'

In [13]:
cat_selector = '#QD8sx38qa > div.c3cb2d10 > div > div.hH5f5Do0 > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(1) > div > ul'
lis = soup.select(f'{cat_selector} > li')
for li in lis:
    print(li.get_text().strip())

헤어질 결심
영화 줄거리


#QD8sx38qa > div.c3cb2d10 > div > div.hH5f5Do0 > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(13) > div:nth-child(1) > div > div:nth-child(4) > div

#QD8sx38qa > div.c3cb2d10 > div > div.hH5f5Do0 > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(13) > div:nth-child(1) > div > div:nth-child(3) > div

In [14]:
summary_selector3 = '#QD8sx38qa > div.c3cb2d10 > div > div.hH5f5Do0 > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(13) > div:nth-child(1) > div > div:nth-child(3) > div'
summary_selector4 = '#QD8sx38qa > div.c3cb2d10 > div > div.hH5f5Do0 > div:nth-child(5) > div > div:nth-child(2) > div > div > div:nth-child(13) > div:nth-child(1) > div > div:nth-child(4) > div'
try:
    summary = soup.select_one(summary_selector3).get_text().strip()
except:
    try:
        summary = soup.select_one(summary_selector4).get_text().strip()
    except:
        summary = ''
print(summary)

1. 개요2. 전반부 (부산)3. 후반부 (이포)


In [62]:
titles, categories, summaries = [], [], []
for page in page_urls:
    driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
    driver.get(page)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.select_one('.hH5f5Do0').find('h1').get_text().strip()
    titles.append(title)
    lis = soup.select(f'{cat_selector} > li')
    for li in lis:
        categories.append(li.get_text().strip())

    try:
        summary = soup.select_one(summary_selector3).get_text().strip()
    except:
        try:
            summary = soup.select_one(summary_selector4).get_text().strip()
        except:
            summary = ''
    summaries.append(summary)  
    driver.close()
    time.sleep(1)

In [63]:
len(titles), len(categories), len(summaries)

(72, 247, 72)

In [65]:
summaries[:5]

['2023년 4월부터 진행한 대규모 스트리머 서버인 악어의 놀이터의 진행과정을 담은 문서이다.',
 '욕심쟁이 오리아저씨를 원작으로 하는 게임들을 설명하는 항목.',
 '',
 '',
 '역대 베니스 국제 영화제   은사자상   제50회(1993년) →  제51회(1994년) →  제52회(1995년) 코쉬 바 코쉬→ 천상의 피조물리틀 오데사일 토로→ 폐지']